# Neural Fingerprint Solubility result

In [21]:
# Example regression script using neural fingerprints.
#
# Compares Morgan fingerprints to neural fingerprints.

import autograd.numpy as np
import autograd.numpy.random as npr

from neuralfingerprint import load_data
from neuralfingerprint import build_morgan_deep_net
from neuralfingerprint import build_conv_deep_net
from neuralfingerprint import normalize_array, adam
from neuralfingerprint import build_batched_grad
from neuralfingerprint.util import rmse

from autograd import grad

task_params = {'target_name' : 'measured log solubility in mols per litre',
               'data_file'   : 'delaney.csv'}
N_train = 800
N_val   = 20
N_test  = 20

model_params = dict(fp_length=50,    # Usually neural fps need far fewer dimensions than morgan.
                    fp_depth=6,      # The depth of the network equals the fingerprint radius.
                    conv_width=20,   # Only the neural fps need this parameter.
                    h1_size=100,     # Size of hidden layer of network on top of fps.
                    L2_reg=np.exp(-2))
train_params = dict(num_iters=800,
                    batch_size=100,
                    init_scale=np.exp(-4),
                    step_size=np.exp(-6))

# Define the architecture of the network that sits on top of the fingerprints.
vanilla_net_params = dict(
    layer_sizes = [model_params['fp_length'], model_params['h1_size']],  # One hidden layer.
    normalize=True, L2_reg = model_params['L2_reg'], nll_func = rmse)

def train_nn(pred_fun, loss_fun, num_weights, train_smiles, train_raw_targets, train_params, seed=0,
             validation_smiles=None, validation_raw_targets=None):
    """loss_fun has inputs (weights, smiles, targets)"""
    print "Total number of weights in the network:", num_weights
    init_weights = npr.RandomState(seed).randn(num_weights) * train_params['init_scale']

    num_print_examples = 100
    train_targets, undo_norm = normalize_array(train_raw_targets)
    training_curve = []
    def callback(weights, iter):
        if iter % 10 == 0:
            print "max of weights", np.max(np.abs(weights))
            train_preds = undo_norm(pred_fun(weights, train_smiles[:num_print_examples]))
            cur_loss = loss_fun(weights, train_smiles[:num_print_examples], train_targets[:num_print_examples])
            training_curve.append(cur_loss)
            print "Iteration", iter, "loss", cur_loss,\
                  "train RMSE", rmse(train_preds, train_raw_targets[:num_print_examples]),
            if validation_smiles is not None:
                validation_preds = undo_norm(pred_fun(weights, validation_smiles))
                print "Validation RMSE", iter, ":", rmse(validation_preds, validation_raw_targets),

    # Build gradient using autograd.
    grad_fun = grad(loss_fun)
    grad_fun_with_data = build_batched_grad(grad_fun, train_params['batch_size'],
                                            train_smiles, train_targets)

    # Optimize weights.
    trained_weights = adam(grad_fun_with_data, init_weights, callback=callback,
                           num_iters=train_params['num_iters'], step_size=train_params['step_size'])

    def predict_func(new_smiles):
        """Returns to the original units that the raw targets were in."""
        return undo_norm(pred_fun(trained_weights, new_smiles))
    return predict_func, trained_weights, training_curve


def main():
    print "Loading data..."
    traindata, valdata, testdata = load_data(
        task_params['data_file'], (N_train, N_val, N_test),
        input_name='smiles', target_name=task_params['target_name'])
    train_inputs, train_targets = traindata
    val_inputs,   val_targets   = valdata
    test_inputs,  test_targets  = testdata

    def print_performance(pred_func):
        train_preds = pred_func(train_inputs)
        val_preds = pred_func(val_inputs)
        print "\nPerformance (RMSE) on " + task_params['target_name'] + ":"
        print "Train:", rmse(train_preds, train_targets)
        print "Test: ", rmse(val_preds,  val_targets)
        print "-" * 80
        return rmse(val_preds, val_targets)

    def run_morgan_experiment():
        loss_fun, pred_fun, net_parser = \
            build_morgan_deep_net(model_params['fp_length'],
                                  model_params['fp_depth'], vanilla_net_params)
        num_weights = len(net_parser)
        predict_func, trained_weights, conv_training_curve = \
            train_nn(pred_fun, loss_fun, num_weights, train_inputs, train_targets,
                     train_params, validation_smiles=val_inputs, validation_raw_targets=val_targets)
        return print_performance(predict_func)

    def run_conv_experiment():
        conv_layer_sizes = [model_params['conv_width']] * model_params['fp_depth']
        conv_arch_params = {'num_hidden_features' : conv_layer_sizes,
                            'fp_length' : model_params['fp_length'], 'normalize' : 1}
        loss_fun, pred_fun, conv_parser = \
            build_conv_deep_net(conv_arch_params, vanilla_net_params, model_params['L2_reg'])
        num_weights = len(conv_parser)
        predict_func, trained_weights, conv_training_curve = \
            train_nn(pred_fun, loss_fun, num_weights, train_inputs, train_targets,
                     train_params, validation_smiles=val_inputs, validation_raw_targets=val_targets)
        test_predictions = predict_func(test_inputs)
        return rmse(test_predictions, test_targets)

    print "Task params", task_params
    print
    print "Starting Morgan fingerprint experiment..."
    test_loss_morgan = run_morgan_experiment()
    print "Starting neural fingerprint experiment..."
    test_loss_neural = run_conv_experiment()
    print
    print "Morgan test RMSE:", test_loss_morgan, "Neural test RMSE:", test_loss_neural

if __name__ == '__main__':
    main()


Loading data...
Task params {'target_name': 'measured log solubility in mols per litre', 'data_file': 'delaney.csv'}

Starting Morgan fingerprint experiment...
Total number of weights in the network: 5201
max of weights 0.06962983567500523
Iteration 0 loss 1.0681550956230024 train RMSE 2.2262093968771457 Validation RMSE 0 : 2.5194018852922193 max of weights 0.07909914663176974
Iteration 10 loss 0.9749391873591123 train RMSE 2.0318360131424322 Validation RMSE 10 : 2.281904008238235 max of weights 0.0966915817527744
Iteration 20 loss 0.8955948789425274 train RMSE 1.8662927335768003 Validation RMSE 20 : 2.0896241502453066 max of weights 0.1159570060218332
Iteration 30 loss 0.8286966024806365 train RMSE 1.726778759008268 Validation RMSE 30 : 1.9473471235843123 max of weights 0.12893227439072727
Iteration 40 loss 0.7670027963527982 train RMSE 1.5980900498437653 Validation RMSE 40 : 1.8062391535848397 max of weights 0.1398175695778033
Iteration 50 loss 0.7415104899426724 train RMSE 1.5448348

Iteration 560 loss 0.17035132784767495 train RMSE 0.35016633345118026 Validation RMSE 560 : 1.7010086923055947 max of weights 0.7315450763488113
Iteration 570 loss 0.12901928984381208 train RMSE 0.2639813332488222 Validation RMSE 570 : 1.6694959391218331 max of weights 0.7356953376594595
Iteration 580 loss 0.16225667451621806 train RMSE 0.33322024942168205 Validation RMSE 580 : 1.6442775620411383 max of weights 0.7381066295062453
Iteration 590 loss 0.18482287273122106 train RMSE 0.3802212719850046 Validation RMSE 590 : 1.663453658380517 max of weights 0.7389376570466474
Iteration 600 loss 0.15566803231868245 train RMSE 0.31942569859777936 Validation RMSE 600 : 1.6555307538734039 max of weights 0.7413988281981054
Iteration 610 loss 0.14167916908206007 train RMSE 0.2902390980190949 Validation RMSE 610 : 1.6638955741749661 max of weights 0.7432117087449247
Iteration 620 loss 0.12241926393092051 train RMSE 0.2500580915412507 Validation RMSE 620 : 1.6530758520527415 max of weights 0.7461883

# Neural Fingerprint photovoltaic efficiency result

In [3]:
# Example regression script using neural fingerprints.
#
# Compares Morgan fingerprints to neural fingerprints.

import autograd.numpy as np
import autograd.numpy.random as npr

from neuralfingerprint import load_data
from neuralfingerprint import build_morgan_deep_net
from neuralfingerprint import build_conv_deep_net
from neuralfingerprint import normalize_array, adam
from neuralfingerprint import build_batched_grad
from neuralfingerprint.util import rmse

from autograd import grad

task_params = {'target_name' : 'PCE',
               'data_file'   : 'cep-processed.csv'}
N_train = 19800
N_val   = 100
N_test  = 100

model_params = dict(fp_length=50,    # Usually neural fps need far fewer dimensions than morgan.
                    fp_depth=6,      # The depth of the network equals the fingerprint radius.
                    conv_width=20,   # Only the neural fps need this parameter.
                    h1_size=100,     # Size of hidden layer of network on top of fps.
                    L2_reg=np.exp(-2))
train_params = dict(num_iters=900,
                    batch_size=100,
                    init_scale=np.exp(-4),
                    step_size=np.exp(-6))

# Define the architecture of the network that sits on top of the fingerprints.
vanilla_net_params = dict(
    layer_sizes = [model_params['fp_length'], model_params['h1_size']],  # One hidden layer.
    normalize=True, L2_reg = model_params['L2_reg'], nll_func = rmse)

def train_nn(pred_fun, loss_fun, num_weights, train_smiles, train_raw_targets, train_params, seed=0,
             validation_smiles=None, validation_raw_targets=None):
    """loss_fun has inputs (weights, smiles, targets)"""
    print "Total number of weights in the network:", num_weights
    init_weights = npr.RandomState(seed).randn(num_weights) * train_params['init_scale']

    num_print_examples = 100
    train_targets, undo_norm = normalize_array(train_raw_targets)
    training_curve = []
    def callback(weights, iter):
        if iter % 10 == 0:
            print "max of weights", np.max(np.abs(weights))
            train_preds = undo_norm(pred_fun(weights, train_smiles[:num_print_examples]))
            cur_loss = loss_fun(weights, train_smiles[:num_print_examples], train_targets[:num_print_examples])
            training_curve.append(cur_loss)
            print "Iteration", iter, "loss", cur_loss,\
                  "train RMSE", rmse(train_preds, train_raw_targets[:num_print_examples]),
            if validation_smiles is not None:
                validation_preds = undo_norm(pred_fun(weights, validation_smiles))
                print "Validation RMSE", iter, ":", rmse(validation_preds, validation_raw_targets),

    # Build gradient using autograd.
    grad_fun = grad(loss_fun)
    grad_fun_with_data = build_batched_grad(grad_fun, train_params['batch_size'],
                                            train_smiles, train_targets)

    # Optimize weights.
    trained_weights = adam(grad_fun_with_data, init_weights, callback=callback,
                           num_iters=train_params['num_iters'], step_size=train_params['step_size'])

    def predict_func(new_smiles):
        """Returns to the original units that the raw targets were in."""
        return undo_norm(pred_fun(trained_weights, new_smiles))
    return predict_func, trained_weights, training_curve


def main():
    print "Loading data..."
    traindata, valdata, testdata = load_data(
        task_params['data_file'], (N_train, N_val, N_test),
        input_name='smiles', target_name=task_params['target_name'])
    train_inputs, train_targets = traindata
    val_inputs,   val_targets   = valdata
    test_inputs,  test_targets  = testdata

    def print_performance(pred_func):
        train_preds = pred_func(train_inputs)
        val_preds = pred_func(val_inputs)
        print "\nPerformance (RMSE) on " + task_params['target_name'] + ":"
        print "Train:", rmse(train_preds, train_targets)
        print "Test: ", rmse(val_preds,  val_targets)
        print "-" * 80
        return rmse(val_preds, val_targets)

    def run_morgan_experiment():
        loss_fun, pred_fun, net_parser = \
            build_morgan_deep_net(model_params['fp_length'],
                                  model_params['fp_depth'], vanilla_net_params)
        num_weights = len(net_parser)
        predict_func, trained_weights, conv_training_curve = \
            train_nn(pred_fun, loss_fun, num_weights, train_inputs, train_targets,
                     train_params, validation_smiles=val_inputs, validation_raw_targets=val_targets)
        return print_performance(predict_func)

    def run_conv_experiment():
        conv_layer_sizes = [model_params['conv_width']] * model_params['fp_depth']
        conv_arch_params = {'num_hidden_features' : conv_layer_sizes,
                            'fp_length' : model_params['fp_length'], 'normalize' : 1}
        loss_fun, pred_fun, conv_parser = \
            build_conv_deep_net(conv_arch_params, vanilla_net_params, model_params['L2_reg'])
        num_weights = len(conv_parser)
        predict_func, trained_weights, conv_training_curve = \
            train_nn(pred_fun, loss_fun, num_weights, train_inputs, train_targets,
                     train_params, validation_smiles=val_inputs, validation_raw_targets=val_targets)
        test_predictions = predict_func(test_inputs)
        return rmse(test_predictions, test_targets)

    print "Task params", task_params
    print
    print "Starting Morgan fingerprint experiment..."
    test_loss_morgan = run_morgan_experiment()
    print "Starting neural fingerprint experiment..."
    test_loss_neural = run_conv_experiment()
    print
    print "Morgan test RMSE:", test_loss_morgan, "Neural test RMSE:", test_loss_neural

if __name__ == '__main__':
    main()


Loading data...
Task params {'target_name': 'PCE', 'data_file': 'cep-processed.csv'}

Starting Morgan fingerprint experiment...
Total number of weights in the network: 5201
max of weights 0.06962983567500523
Iteration 0 loss 0.9385766593076806 train RMSE 2.376559308764014 Validation RMSE 0 : 2.3435034177697256 max of weights 0.07283979395826158
Iteration 10 loss 0.9160849416948444 train RMSE 2.3195707149312503 Validation RMSE 10 : 2.3076133365618023 max of weights 0.09576515212845581
Iteration 20 loss 0.8888843114093663 train RMSE 2.250610964861513 Validation RMSE 20 : 2.248498139613563 max of weights 0.11674110323273869
Iteration 30 loss 0.8486465259682635 train RMSE 2.1485791633614855 Validation RMSE 30 : 2.165018351769259 max of weights 0.13085101807156044
Iteration 40 loss 0.8167243858342416 train RMSE 2.0675849578779233 Validation RMSE 40 : 2.113094017779867 max of weights 0.1439236181565618
Iteration 50 loss 0.8025603580531296 train RMSE 2.031602155485496 Validation RMSE 50 : 2.0

Iteration 610 loss 0.7936468229372441 train RMSE 2.0075077730175317 Validation RMSE 610 : 2.057941103562591 max of weights 0.35430627038374407
Iteration 620 loss 0.7984533760710435 train RMSE 2.0196418663839326 Validation RMSE 620 : 2.0735598670118005 max of weights 0.35042787262506037
Iteration 630 loss 0.7885608730289505 train RMSE 1.9945427031387322 Validation RMSE 630 : 2.0682977902693076 max of weights 0.343881935661157
Iteration 640 loss 0.7783651903182461 train RMSE 1.968664297171954 Validation RMSE 640 : 2.0663816850928995 max of weights 0.34882295665066954
Iteration 650 loss 0.7723875693934842 train RMSE 1.9534994711541878 Validation RMSE 650 : 2.0654812922348142 max of weights 0.3566732318475272
Iteration 660 loss 0.7768495146059214 train RMSE 1.9647936665598855 Validation RMSE 660 : 2.062648357678569 max of weights 0.3616312021603846
Iteration 670 loss 0.7908215083586304 train RMSE 2.000149594377394 Validation RMSE 670 : 2.079394086776996 max of weights 0.37158096409915525
I

Iteration 270 loss 0.6269020807459134 train RMSE 1.5849836693245236 Validation RMSE 270 : 1.315641751041517 max of weights 0.4399137455189865
Iteration 280 loss 0.6279361234405393 train RMSE 1.5876054777068718 Validation RMSE 280 : 1.3350412226419421 max of weights 0.46149638158946704
Iteration 290 loss 0.6193940875009991 train RMSE 1.565950083237545 Validation RMSE 290 : 1.349455616240558 max of weights 0.469041011566497
Iteration 300 loss 0.6404061861305668 train RMSE 1.619097061700056 Validation RMSE 300 : 1.2795156593651718 max of weights 0.4868191313175739
Iteration 310 loss 0.6421527264390964 train RMSE 1.6235640949950436 Validation RMSE 310 : 1.378492630203372 max of weights 0.5089146186937037
Iteration 320 loss 0.657551259493034 train RMSE 1.6624647429091761 Validation RMSE 320 : 1.3578713336536172 max of weights 0.5277828525667964
Iteration 330 loss 0.6287664089874075 train RMSE 1.5895723346756039 Validation RMSE 330 : 1.350203655621367 max of weights 0.5427490688882388
Iterat

Iteration 840 loss 0.5927301887628121 train RMSE 1.4971068537419625 Validation RMSE 840 : 1.4317073092857386 max of weights 1.3173575915439493
Iteration 850 loss 0.5850127762203887 train RMSE 1.4775485927852683 Validation RMSE 850 : 1.411391041551979 max of weights 1.323692778810133
Iteration 860 loss 0.6128828003584331 train RMSE 1.548109651034858 Validation RMSE 860 : 1.493185303198071 max of weights 1.3308902763500237
Iteration 870 loss 0.5828216160150949 train RMSE 1.4719710080488257 Validation RMSE 870 : 1.361045387898216 max of weights 1.3496007478019791
Iteration 880 loss 0.5876770913121582 train RMSE 1.4842265922954168 Validation RMSE 880 : 1.40577851583553 max of weights 1.3546092784933141
Iteration 890 loss 0.5820578098932715 train RMSE 1.4699499774958606 Validation RMSE 890 : 1.3344720231956197
Morgan test RMSE: 2.0830770322561856 Neural test RMSE: 1.6761424673542065


# Neural Fingerprint drug efficacy result

In [16]:
# Example regression script using neural fingerprints.
#
# Compares Morgan fingerprints to neural fingerprints.

import autograd.numpy as np
import autograd.numpy.random as npr

from neuralfingerprint import load_data
from neuralfingerprint import build_morgan_deep_net
from neuralfingerprint import build_conv_deep_net
from neuralfingerprint import normalize_array, adam
from neuralfingerprint import build_batched_grad
from neuralfingerprint.util import rmse

from autograd import grad

task_params = {'target_name' : 'activity',
               'data_file'   : 'malaria-processed.csv'}
N_train = 9700
N_val   = 100
N_test  = 100

model_params = dict(fp_length=50,    # Usually neural fps need far fewer dimensions than morgan.
                    fp_depth=6,      # The depth of the network equals the fingerprint radius.
                    conv_width=20,   # Only the neural fps need this parameter.
                    h1_size=100,     # Size of hidden layer of network on top of fps.
                    L2_reg=np.exp(-2))
train_params = dict(num_iters=800,
                    batch_size=100,
                    init_scale=np.exp(-4),
                    step_size=np.exp(-6))

# Define the architecture of the network that sits on top of the fingerprints.
vanilla_net_params = dict(
    layer_sizes = [model_params['fp_length'], model_params['h1_size']],  # One hidden layer.
    normalize=True, L2_reg = model_params['L2_reg'], nll_func = rmse)

def train_nn(pred_fun, loss_fun, num_weights, train_smiles, train_raw_targets, train_params, seed=0,
             validation_smiles=None, validation_raw_targets=None):
    """loss_fun has inputs (weights, smiles, targets)"""
    print "Total number of weights in the network:", num_weights
    init_weights = npr.RandomState(seed).randn(num_weights) * train_params['init_scale']

    num_print_examples = 100
    train_targets, undo_norm = normalize_array(train_raw_targets)
    training_curve = []
    def callback(weights, iter):
        if iter % 10 == 0:
            print "max of weights", np.max(np.abs(weights))
            train_preds = undo_norm(pred_fun(weights, train_smiles[:num_print_examples]))
            cur_loss = loss_fun(weights, train_smiles[:num_print_examples], train_targets[:num_print_examples])
            training_curve.append(cur_loss)
            print "Iteration", iter, "loss", cur_loss,\
                  "train RMSE", rmse(train_preds, train_raw_targets[:num_print_examples]),
            if validation_smiles is not None:
                validation_preds = undo_norm(pred_fun(weights, validation_smiles))
                print "Validation RMSE", iter, ":", rmse(validation_preds, validation_raw_targets),

    # Build gradient using autograd.
    grad_fun = grad(loss_fun)
    grad_fun_with_data = build_batched_grad(grad_fun, train_params['batch_size'],
                                            train_smiles, train_targets)

    # Optimize weights.
    trained_weights = adam(grad_fun_with_data, init_weights, callback=callback,
                           num_iters=train_params['num_iters'], step_size=train_params['step_size'])

    def predict_func(new_smiles):
        """Returns to the original units that the raw targets were in."""
        return undo_norm(pred_fun(trained_weights, new_smiles))
    return predict_func, trained_weights, training_curve


def main():
    print "Loading data..."
    traindata, valdata, testdata = load_data(
        task_params['data_file'], (N_train, N_val, N_test),
        input_name='smiles', target_name=task_params['target_name'])
    train_inputs, train_targets = traindata
    val_inputs,   val_targets   = valdata
    test_inputs,  test_targets  = testdata

    def print_performance(pred_func):
        train_preds = pred_func(train_inputs)
        val_preds = pred_func(val_inputs)
        print "\nPerformance (RMSE) on " + task_params['target_name'] + ":"
        print "Train:", rmse(train_preds, train_targets)
        print "Test: ", rmse(val_preds,  val_targets)
        print "-" * 80
        return rmse(val_preds, val_targets)

    def run_morgan_experiment():
        loss_fun, pred_fun, net_parser = \
            build_morgan_deep_net(model_params['fp_length'],
                                  model_params['fp_depth'], vanilla_net_params)
        num_weights = len(net_parser)
        predict_func, trained_weights, conv_training_curve = \
            train_nn(pred_fun, loss_fun, num_weights, train_inputs, train_targets,
                     train_params, validation_smiles=val_inputs, validation_raw_targets=val_targets)
        return print_performance(predict_func)

    def run_conv_experiment():
        conv_layer_sizes = [model_params['conv_width']] * model_params['fp_depth']
        conv_arch_params = {'num_hidden_features' : conv_layer_sizes,
                            'fp_length' : model_params['fp_length'], 'normalize' : 1}
        loss_fun, pred_fun, conv_parser = \
            build_conv_deep_net(conv_arch_params, vanilla_net_params, model_params['L2_reg'])
        num_weights = len(conv_parser)
        predict_func, trained_weights, conv_training_curve = \
            train_nn(pred_fun, loss_fun, num_weights, train_inputs, train_targets,
                     train_params, validation_smiles=val_inputs, validation_raw_targets=val_targets)
        test_predictions = predict_func(test_inputs)
        return rmse(test_predictions, test_targets)

    print "Task params", task_params
    print
    print "Starting Morgan fingerprint experiment..."
    test_loss_morgan = run_morgan_experiment()
    print "Starting neural fingerprint experiment..."
    test_loss_neural = run_conv_experiment()
    print
    print "Morgan test RMSE:", test_loss_morgan, "Neural test RMSE:", test_loss_neural

if __name__ == '__main__':
    main()


Loading data...
Task params {'target_name': 'activity', 'data_file': 'malaria-processed.csv'}

Starting Morgan fingerprint experiment...
Total number of weights in the network: 5201
max of weights 0.06962983567500523
Iteration 0 loss 0.819681257416317 train RMSE 0.9969002045258831 Validation RMSE 0 : 1.2043149332577938 max of weights 0.07764954491251569
Iteration 10 loss 0.8092990967412906 train RMSE 0.9842587942540298 Validation RMSE 10 : 1.197878260767718 max of weights 0.0826609586957519
Iteration 20 loss 0.8056825758001497 train RMSE 0.9798403579576411 Validation RMSE 20 : 1.1962065000125355 max of weights 0.09842937474201935
Iteration 30 loss 0.7986581402212991 train RMSE 0.9712692654347401 Validation RMSE 30 : 1.1935355786123156 max of weights 0.10905974230467713
Iteration 40 loss 0.792582788066391 train RMSE 0.9638492276613394 Validation RMSE 40 : 1.194161178458829 max of weights 0.12172841950819184
Iteration 50 loss 0.7889750401098654 train RMSE 0.9594404320765614 Validation RM

Iteration 570 loss 0.746800384653625 train RMSE 0.9071060356634002 Validation RMSE 570 : 1.1717394464558863 max of weights 0.43554205152874464
Iteration 580 loss 0.7461619737608755 train RMSE 0.9063075474934804 Validation RMSE 580 : 1.1632729431326834 max of weights 0.4295428789210161
Iteration 590 loss 0.734579441449931 train RMSE 0.8921887275450894 Validation RMSE 590 : 1.1672547126098989 max of weights 0.43199149606044523
Iteration 600 loss 0.7369490589954658 train RMSE 0.8950406350463174 Validation RMSE 600 : 1.1835877038304337 max of weights 0.4325035724910027
Iteration 610 loss 0.7380291860466626 train RMSE 0.8963193608339377 Validation RMSE 610 : 1.1907861927036356 max of weights 0.4424381897056064
Iteration 620 loss 0.7364815887260585 train RMSE 0.8944016720941563 Validation RMSE 620 : 1.1960273893620104 max of weights 0.4580562430347485
Iteration 630 loss 0.7411658312135444 train RMSE 0.90008177643049 Validation RMSE 630 : 1.2054543627927945 max of weights 0.4612045074572156
I